In [2]:
%run "F:\\onedrive\\Desktop\\DATA SCIENCE\\Project 1\\Cross-Market Analysis\\bootstrap.py"

In [3]:
from src.config.settings import (Asset_Date_Ranges)
import pandas as pd

In [5]:
df = pd.read_csv("F:\\onedrive\\Desktop\\DATA SCIENCE\\Project 1\\Cross-Market Analysis\\Cross-Market Data\\raw\\oil\\oil_prices.csv")

# Parse date column

In [6]:
df.head()

,Unnamed: 0,Date,Price
0,0,1986-01-02,25.56
1,1,1986-01-03,26.00
2,2,1986-01-06,26.53
3,3,1986-01-07,25.85
4,4,1986-01-08,25.87


In [7]:
# Check the data type
df.dtypes

Unnamed: 0      int64
Date           object
Price         float64
dtype: object

In [8]:
# Parse internally as day first for universal application from settings.py.Now the system will read the column as date.
df["Date"] = pd.to_datetime(df["Date"], format = "%Y-%m-%d")
df.dtypes

Unnamed: 0             int64
Date          datetime64[ns]
Price                float64
dtype: object

# Filter 2020–2026

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10081 entries, 0 to 10080
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Unnamed: 0  10081 non-null  int64         
 1   Date        10081 non-null  datetime64[ns]
 2   Price       10081 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 236.4 KB


In [10]:
df.drop('Unnamed: 0', axis = 1, inplace = True)

In [11]:
oil_start = pd.to_datetime(Asset_Date_Ranges['oil']['start'], dayfirst = True)
oil_end = pd.to_datetime(Asset_Date_Ranges['oil']['end'], dayfirst = True)
df = df[df['Date'].between(oil_start, oil_end)]
print(df.head())
print(df.tail())

           Date  Price
8569 2020-01-02  61.17
8570 2020-01-03  63.00
8571 2020-01-06  63.27
8572 2020-01-07  62.70
8573 2020-01-08  59.65
            Date  Price
10076 2026-01-13  60.85
10077 2026-01-14  61.84
10078 2026-01-15  59.13
10079 2026-01-16  59.40
10080 2026-01-20  60.30


# Handle missing days

In [12]:
df.isnull().sum()

Date     0
Price    0
dtype: int64

In [13]:
df.columns

Index(['Date', 'Price'], dtype='object')

In [14]:
# Sort the values as per date
df.sort_values(by = 'Date', ascending = True, inplace = True)
print(df.head())
len(df)

           Date  Price
8569 2020-01-02  61.17
8570 2020-01-03  63.00
8571 2020-01-06  63.27
8572 2020-01-07  62.70
8573 2020-01-08  59.65


1512

In [15]:
# Check for missing days, specifying the frequency as D indicates total calender dates.
expected_dates = pd.date_range(start = df['Date'].min(), end = df['Date'].max(), freq = 'D')
missing_days = expected_dates.difference(df['Date'])
len(missing_days)

699

In [16]:
# Display the date values to full calender with NaN in missing prices. This is done so when cross-analysis with crypto takes place the data should not lie.
df = df.set_index('Date').reindex(expected_dates)
df.head()

,Price
2020-01-02,61.17
2020-01-03,63.00
2020-01-04,NaN
2020-01-05,NaN
2020-01-06,63.27


In [17]:
# Forward Fill Prices as when the market is closed the prices does not change.
df['Price'] = df['Price'].ffill()
df.head()

,Price
2020-01-02,61.17
2020-01-03,63.00
2020-01-04,63.00
2020-01-05,63.00
2020-01-06,63.27


In [18]:
# Restore the date column
df = df.reset_index().rename(columns = {'index' : 'Date'})
df.head()

,Date,Price
0,2020-01-02,61.17
1,2020-01-03,63.00
2,2020-01-04,63.00
3,2020-01-05,63.00
4,2020-01-06,63.27


# Rename columns

In [19]:
df = df.rename(columns = {'Price' : 'price_usd'})
df = df.rename(columns = {'Date' : 'date'})
df.head()

,date,price_usd
0,2020-01-02,61.17
1,2020-01-03,63.00
2,2020-01-04,63.00
3,2020-01-05,63.00
4,2020-01-06,63.27


In [20]:
sum(df['date'].duplicated())

0

# Save cleaned dataset

In [101]:
df.to_csv("F:\\onedrive\\Desktop\\DATA SCIENCE\\Project 1\\Cross-Market Analysis\\Cross-Market Data\\cleaned\\oil\\oil_prices.csv")